In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np
from PIL import Image
from utils import read_mat_file, visualize_sample_dataset, vae_loss
from dataloader import SVHDDataset
import scipy.io

from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from model import VGG
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm import tqdm
from torch.utils.data import DataLoader, random_split
import os
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
# Train the model
from model import VGG16, VAE
import cv2 

os.environ['TORCH_NNPACK'] = '0'

In [2]:
train_path = '/data/omscs_datasets/train/'
train_mat_file_path = train_path + 'digitStruct.mat'
test_path = '/data/omscs_datasets/train/'
test_mat_file_path = test_path + 'digitStruct.mat'
device = torch.device("cuda")

In [33]:
def calculate_anchor_sizes(image, num_sizes=3, size_ratios=[1, 0.75, 0.5]):
    height, width, _ = image.shape

    # 이미지 크기에 기반한 기본 앵커 크기 계산
    base_size = min(height, width) // 4  # 예시: 가로, 세로 중 작은 쪽의 1/4

    # 여러 크기와 비율의 앵커 박스 계산
    anchor_sizes = []
    for i in range(1, num_sizes + 1):
        size = base_size * i
        for ratio in size_ratios:
            anchor_width = int(size * ratio)
            anchor_height = int(size * (1 / ratio))
            anchor_sizes.append((anchor_width, anchor_height))

    return anchor_sizes


def detect_digit_from_anchor_image_reconstruct(img_path, model,vgg16, output_image_path, device, num):
    # Load your model
    #model.load_state_dict(torch.load('best_model.pth'))
    model.eval()

    # Load the image (now in color)
    image = cv2.imread(img_path)

    # Define the window size and step size
    win_size = 16
    window_size = (win_size, win_size)
    print('image:', image.shape)
    window_size = (int(image.shape[0]//5), int(image.shape[1]//5))
    step_size = 10

    # List to store bounding box coordinates and predictions
    bbox_coords = []
    final_bbox_coords = []
    losses_array = []
    predicted_arr = []
    red_arr = []

    anchor_sizes = calculate_anchor_sizes(image)
    anchor_ratios = [1, 0.75, 0.5]

    # Sliding window
    for y in range(0, image.shape[0] - window_size[1], step_size):
        for x in range(0, image.shape[1] - window_size[0], step_size):
            for size in anchor_sizes:
                for ratio in anchor_ratios:
                    # calculate anchor box dimensions
                    anchor_width = int(size[0] * ratio)
                    anchor_height = int(size[1] * (1 / ratio))

                     # Calculate coordinates of the window
                    x1 = max(x - anchor_width // 2, 0)
                    y1 = max(y - anchor_height // 2, 0)
                    x2 = min(x1 + anchor_width, image.shape[1])
                    y2 = min(y1 + anchor_height, image.shape[0])

                    # Extract and preprocess the window
                    #window = image[y:y + window_size[1], x:x + window_size[0]]
                    window = image[y1:y2, x1:x2]
                    processed_window = cv2.resize(window, (32, 32))
                    processed_window = transforms.ToTensor()(processed_window)
                    #processed_window = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(processed_window)
                    processed_window = processed_window.unsqueeze(0).to(torch.float32).to(device)
                    #print('processed_window:', processed_window.shape)
                    # Classify the window
                    with torch.no_grad():
                        x_reconstructed, mu, log_var  = model(processed_window)
                        loss = vae_loss(x_reconstructed, processed_window, mu, log_var)
                        losses_array.append(loss.item())
                        #_, predicted = torch.max(outputs, 1)
                        bbox_coords.append((x, y, x + window_size[0], y + window_size[1], loss.item()))

                        with torch.no_grad():
                            outputs = vgg16(processed_window)
                            _, predicted = torch.max(outputs, 1)
                            predicted_arr.append(predicted.item())
                        

    sorted_lst_asc = sorted(losses_array)[::-1]
    limit_loss = sorted_lst_asc[num]
    for bbox, lss, pred in zip(bbox_coords, losses_array, predicted_arr):
        if lss > limit_loss:
            final_bbox_coords.append(bbox)
            red_arr.append(pred)
    # Draw bounding boxes and text on the original image
    for (x1, y1, x2, y2, prediction), pred_digit in zip(final_bbox_coords, red_arr):
        print('prediction:',pred_digit)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 1)
        cv2.putText(image, str(pred_digit), (x1, y1+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # Save the final image with bounding boxes and text
    cv2.imwrite(output_image_path, image)

In [34]:
# Hyperparameters
image_channels = 3
h_dim = 1024
z_dim = 32
learning_rate = 1e-3

vae = VAE(image_channels=image_channels, h_dim=h_dim, z_dim=z_dim)
vae = vae.to(device)
vae.load_state_dict(torch.load('weights/vae_best_model.pth'))

<All keys matched successfully>

In [35]:
vgg16 =  VGG16(num_classes=11).to(device)
vgg16.load_state_dict(torch.load('weights/best_model.pth'))

<All keys matched successfully>

In [36]:
detect_digit_from_image_reconstruct('test_images/test4.png', vae, vgg16, 'output_with_boxes_reconstruct.png', device, num=8)

image: (38, 99, 3)
prediction: 1
prediction: 1
prediction: 1
prediction: 1
prediction: 1
prediction: 1
prediction: 1
prediction: 1
